In [1]:
from vertexai.preview.language_models import TextGenerationModel
import pandas as pd

In [21]:
def gen_solution(logs = "", data = ""):
    parameters = {
        "temperature": 0.3,
        "max_output_tokens": 1024,
        "top_p": .8,
        "top_k": 40,
    }
    model = TextGenerationModel.from_pretrained("text-bison@001")

    prompt = """
    背景:現在有一個在Google Cloud Run上運行的應用服務。這個應用服務應該能夠穩定地處理客戶請求，並保持良好的性能。
    角色:你是一位監測此應用服務的數據的AI助理。
        數據類型說明：
        - 日誌訊息:包含有關應用運行狀態的信息。
        - 度量參數:包括CPU使用率、記憶體使用率、響應時間和錯誤率等指標。
        數據可能包含日誌訊息及數個度量參數，或是僅包含其中一項。
    任務：
        1.根據給定的數據，找出出現問題的部分。
        2.根據出現問題的部分，詳細描述這些問題，並提供解決這些問題的方法。
            a. 問題描述：[問題的具體描述]
            b. 可能原因：[問題可能的原因]
            c. 解決方案：[解決問題的建議方案]
        
        請以以上格式回答。
    """


    
    combined_prompt = f"""
        日誌訊息：
        {logs}

        度量參數：
        {data}
        
        {prompt}
    """
    

    response = model.predict(
        combined_prompt,
        **parameters,
    )

    print(f"Response from Model: \n{response.text}")

In [ ]:
def check_abnormalities(logs: str):
    parameters = {
        "temperature": 0.8,
        # "max_output_tokens": 1024,
        "top_p": .8,
        "top_k": 40,
        # "logprobs": 1,
        "candidate_count": 3
    }
    model = TextGenerationModel.from_pretrained("text-bison@001")

    prompt = """
    背景:現在有一個在Google Cloud Run上運行的應用服務。這個應用服務應該能夠穩定地處理客戶請求，並保持良好的性能。
    角色:你是一位監測此應用服務的數據的AI助理。
        數據類型說明：
        - 日誌訊息:包含有關應用運行狀態的信息。
        - 度量參數:包括CPU使用率、記憶體使用率、響應時間和錯誤率等指標。
        數據可能包含日誌訊息及數個度量參數，或是僅包含其中一項。
    任務：
        1.根據給定的數據，分析這些數據來判斷系統是否發生問題。
            - 如果系統沒有發生問題，回答: 0
            - 如果系統有發生問題，回答: 1
        
        請以以上格式回答。
    """

    # combined_prompt = f"{prompt}\n數據:\n{logs}"
    combined_prompt = f"數據:\n{logs}\n\n\n{prompt}"

    response = model.predict(
        # prompt,
        combined_prompt,
        **parameters,
    )

    print(f"Response from Model: \n{response.text}")

In [ ]:
def find_out_abnormalities(logs: str):
    parameters = {
        "temperature": 0.1,
        "max_output_tokens": 1024,
        "top_p": .8,
        "top_k": 40,
        # "logprobs": 1,
        # "candidate_count": 3
    }
    model = TextGenerationModel.from_pretrained("text-bison@002")

    prompt = """
    背景:現在有一個在Google Cloud Run上運行的應用服務。這個應用服務應該能夠穩定地處理客戶請求，並保持良好的性能。
    角色:你是一位監測此應用服務的日誌訊息的AI助理。
    任務:根據給定的日誌訊息，截取發生問題的部分，並包括其上下文的日誌訊息。
    
    在這個任務中，請注意將問題日誌訊息及其上下文整合在一起，而不是分開來處理。
    """

    # combined_prompt = f"{prompt}\n數據:\n{logs}"
    combined_prompt = f"數據:\n{logs}\n\n\n{prompt}"

    response = model.predict(
        # prompt,
        combined_prompt,
        **parameters,
    )

    print(f"Response from Model: \n{response.text}")
    
    return response.text

In [ ]:
logs = """
[2024-01-22 20:13:20.268] [Info] POST /api/users/register application/json:
[2024-01-22 20:13:20.270] [Info] Matched: (register_user) POST /api/users/register application/json
[2024-01-22 20:13:20.273] [Info] Starting user registration: Email: a95424945@gmail.com
[2024-01-22 20:13:21.128] [Error] Inserting user failed with error: Database(SqliteError { code: 2067, message: "UNIQUE constraint failed: Users.email" })
[2024-01-22 20:13:21.131] [Info] Outcome: Failure
[2024-01-22 20:13:21.133] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 20:13:21.135] [Info] Response succeeded.
[2024-01-22 20:13:22.209] [Info] POST /api/users/register application/json:
[2024-01-22 20:13:22.211] [Info] Matched: (register_user) POST /api/users/register application/json
[2024-01-22 20:13:22.213] [Info] Starting user registration: Email: a95424945@gmail.com
[2024-01-22 20:13:23.071] [Error] Inserting user failed with error: Database(SqliteError { code: 2067, message: "UNIQUE constraint failed: Users.email" })
[2024-01-22 20:13:23.072] [Info] Outcome: Failure
[2024-01-22 20:13:23.074] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 20:13:23.076] [Info] Response succeeded.
[2024-01-22 20:13:24.304] [Info] POST /api/users/register application/json:
[2024-01-22 20:13:24.306] [Info] Matched: (register_user) POST /api/users/register application/json
[2024-01-22 20:13:24.308] [Info] Starting user registration: Email: a95424945@gmail.com
[2024-01-22 20:13:25.158] [Error] Inserting user failed with error: Database(SqliteError { code: 2067, message: "UNIQUE constraint failed: Users.email" })
[2024-01-22 20:13:25.160] [Info] Outcome: Failure
[2024-01-22 20:13:25.161] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 20:13:25.163] [Info] Response succeeded.
    """

In [ ]:
logs = """[2024-01-22 20:13:20.268] [Info] POST /api/users/register application/json:
[2024-01-22 20:13:20.270] [Info] Matched: (register_user) POST /api/users/register application/json
[2024-01-22 20:13:20.273] [Info] Starting user registration: Email: a95424945@gmail.com
[2024-01-22 20:13:21.128] [Error] Inserting user failed with error: Database(SqliteError { code: 2067, message: "UNIQUE constraint failed: Users.email" })
[2024-01-22 20:13:21.131] [Info] Outcome: Failure
[2024-01-22 20:13:21.133] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 20:13:21.135] [Info] Response succeeded.
[2024-01-22 20:13:22.209] [Info] POST /api/users/register application/json:
[2024-01-22 20:13:22.211] [Info] Matched: (register_user) POST /api/users/register application/json
[2024-01-22 20:13:22.213] [Info] Starting user registration: Email: a95424945@gmail.com
[2024-01-22 20:13:23.071] [Error] Inserting user failed with error: Database(SqliteError { code: 2067, message: "UNIQUE constraint failed: Users.email" })
[2024-01-22 20:13:23.072] [Info] Outcome: Failure
[2024-01-22 20:13:23.074] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 20:13:23.076] [Info] Response succeeded.
[2024-01-22 20:13:24.304] [Info] POST /api/users/register application/json:
[2024-01-22 20:13:24.306] [Info] Matched: (register_user) POST /api/users/register application/json
[2024-01-22 20:13:24.308] [Info] Starting user registration: Email: a95424945@gmail.com
[2024-01-22 20:13:25.158] [Error] Inserting user failed with error: Database(SqliteError { code: 2067, message: "UNIQUE constraint failed: Users.email" })
[2024-01-22 20:13:25.160] [Info] Outcome: Failure
[2024-01-22 20:13:25.161] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 20:13:25.163] [Info] Response succeeded.
[2024-01-22 20:14:41.430] [Info] GET /api/big_memory/5000:
[2024-01-22 20:14:41.432] [Info] Matched: (big_memory) GET /api/big_memory/<size>
[2024-01-22 20:14:41.433] [Info] Starting to allocate big memory: Size: 5000
[2024-01-22 20:14:41.434] [Info] Completed big memory allocation successfully
[2024-01-22 20:14:41.436] [Info] Outcome: Success
[2024-01-22 20:14:41.438] [Info] Response succeeded.
[2024-01-22 20:14:57.494] [Info] GET /api/big_memory/500000:
[2024-01-22 20:14:57.495] [Info] Matched: (big_memory) GET /api/big_memory/<size>
[2024-01-22 20:14:57.497] [Info] Starting to allocate big memory: Size: 500000
[2024-01-22 20:14:57.501] [Info] Completed big memory allocation successfully
[2024-01-22 20:14:57.503] [Info] Outcome: Success
[2024-01-22 20:14:57.504] [Info] Response succeeded.
[2024-01-22 21:59:48.457] [Info] Starting to show current seats status
[2024-01-22 21:59:48.458] [Error] Checking if the time is within any unavailable timeslots failed with error: PoolClosed
[2024-01-22 21:59:48.460] [Info] Outcome: Failure
[2024-01-22 21:59:48.461] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 21:59:48.463] [Info] Response succeeded."""

In [ ]:
logs = """[2024-01-22 20:14:57.494] [Info] GET /api/big_memory/500000:
[2024-01-22 20:14:57.495] [Info] Matched: (big_memory) GET /api/big_memory/<size>
[2024-01-22 20:14:57.497] [Info] Starting to allocate big memory: Size: 500000
[2024-01-22 20:14:57.501] [Info] Completed big memory allocation successfully
[2024-01-22 20:14:57.503] [Info] Outcome: Success
[2024-01-22 20:14:57.504] [Info] Response succeeded.
[2024-01-22 21:59:48.454] [Info] GET /api/seats/status:
[2024-01-22 21:59:48.455] [Info] Matched: (show_current_seats_status) GET /api/seats/status
[2024-01-22 21:59:48.457] [Info] Starting to show current seats status
[2024-01-22 21:59:48.458] [Error] Checking if the time is within any unavailable timeslots failed with error: PoolClosed
[2024-01-22 21:59:48.460] [Info] Outcome: Failure
[2024-01-22 21:59:48.461] [Warn] Responding with registered (handle_internal_server_error) 500 catcher.
[2024-01-22 21:59:48.463] [Info] Response succeeded."""

In [8]:
cpu_data = pd.read_csv('data/cpu_data.csv')
memory_data = pd.read_csv('data/memory_data.csv')
instance_data = pd.read_csv('data/instance_data.csv')
request_count_data = pd.read_csv('data/request_count_data.csv')
request_latency_data = pd.read_csv('data/request_latency_data.csv')

data_frames = [cpu_data, memory_data, instance_data, request_count_data, request_latency_data]  # 將所有數據集加入列表
data_frames = list(map(lambda df: df.assign(Time=pd.to_datetime(df['Time'])), data_frames))

from functools import reduce

merged_data = reduce(lambda left, right: pd.merge(left, right, on=['Time'], how='outer'), data_frames)

merged_data

,Time,Container CPU Utilization (%),Container Memory Utilization (%),Instance Count (active),Instance Count (idle),Request Count (1xx),Request Count (2xx),Request Count (3xx),Request Count (4xx),Request Latency (ms)
0,2023-12-07 09:00:00,16.0875,53.99,1,1,0,0,0,4,19.416309
1,2023-12-07 09:01:00,16.0875,53.99,1,1,1,0,0,4,323462.632200
2,2023-12-07 09:02:00,16.0875,53.99,1,1,0,0,0,4,17.651190
3,2023-12-07 09:03:00,16.0875,53.99,1,1,0,0,0,4,17.651190
4,2023-12-07 09:04:00,16.0875,53.99,1,1,0,0,0,3,17.667295
...,...,...,...,...,...,...,...,...,...,...
596,2023-12-07 18:56:00,16.0875,48.99,1,1,0,0,0,4,16.090459
597,2023-12-07 18:57:00,16.0875,48.99,1,1,2,0,0,3,323758.031800
598,2023-12-07 18:58:00,16.0875,48.99,1,1,0,0,0,4,323462.632200
599,2023-12-07 18:59:00,16.0875,48.99,1,1,1,0,0,3,323758.031800


In [9]:
cpu_limit = 60.0
memory_limit = 60.0
instance_limit = 2

In [15]:
for index, row in merged_data.iterrows():
    if float(row['Container CPU Utilization (%)']) > cpu_limit or float(row['Container Memory Utilization (%)']) > memory_limit:
        abnormal_data = row
        break

abnormal_data = ', '.join([f"{key}: {value}" for key, value in abnormal_data.items()])    
abnormal_data

'Time: 2023-12-07 09:18:00, Container CPU Utilization (%): 64.8375, Container Memory Utilization (%): 61.99, Instance Count (active): 1, Instance Count (idle): 1, Request Count (1xx): 0, Request Count (2xx): 59, Request Count (3xx): 0, Request Count (4xx): 4, Request Latency (ms): 1176.256346'

In [22]:
# check_abnormalities(logs)
# abnormal_logs = find_out_abnormalities(logs)
abnormal_logs = ""

gen_solution(abnormal_logs, abnormal_data)

Response from Model: 
問題描述：
        容器CPU使用率和記憶體使用率都很高，這可能會導致應用程序性能下降。
    可能原因：
        - 應用程序中的某些進程可能正在使用過多的CPU或記憶體。
        - 應用程序可能正在處理太多的請求。
    解決方案：
        - 檢查應用程序中的進程，並確定哪些進程正在使用過多的CPU或記憶體。
        - 如果應用程序正在處理太多的請求，則可以增加應用程序的容量。


In [4]:
with open('prompt.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# 將文件內容轉換為字符串
prompt = str(file_content)

In [1]:
prompt = """

intents = discord.Intents.all()
client = commands.Bot(command_prefix='!', intents=intents)

@client.event
async def on_ready():
    global discord_channel

    if not discord_channel:
        print('[on_ready] getting discord channel:', DISCORD_CHANNEL_ID)
        discord_channel = client.get_channel(DISCORD_CHANNEL_ID)

    if not discord_channel:
        print("[on_ready] can't access channel:", DISCORD_CHANNEL_ID)
    else:
        print('[on_ready] channel:', discord_channel)
    
    try:
        synced = await client.tree.sync()
        print('[on_ready] synced:', synced)
    except Exception as e:
        print('[on_ready] [ERROR] sync failed:', e)

@client.command()
async def ping(ctx, arg=''):
    print(f'[ping] ping {arg}')
    await ctx.send(f'pong {arg}')

@client.tree.command()
async def slash_ping(interaction):
    print('[slash_ping] ping')
    await interaction.response.send_message('pong')

@client.event
async def on_message(message):
    if message.author == client.user:
        return
    print('[on_message] message.content:', message.content)
    await client.process_commands(message)
----------------------------------------------------------------
使用pytest搭配monkey patch撰寫測試


"""

In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession

# TODO(developer): Update and un-comment below lines
# project_id = "PROJECT_ID"
# location = "us-central1"
# vertexai.init(project=project_id, location=location)

model = GenerativeModel("gemini-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str):
    response = chat.send_message(prompt)
    return response.text

print(get_chat_response(chat, prompt))

```python
import pytest
import discord
from discord.ext import commands

class MockDiscordChannel:
    def __init__(self, id):
        self.id = id

    async def send(self, message):
        print(f"[MockDiscordChannel.send] message: {message}")

class MockDiscordClient:
    def __init__(self):
        self.get_channel = MockDiscordChannel
        self.tree = MockDiscordTree()

class MockDiscordTree:
    async def sync(self):
        print("[MockDiscordTree.sync]")
        return True

@pytest.fixture
def mock_discord_client():
    return MockDiscordClient()

@pytest.fixture
def setup_client(mocker, mock_discord_client):
    global client, discord_channel

    # mock discord client
    mocker.patch("discord.Client", mock_discord_client)
    # mock discord channel
    discord_channel = None

    # create bot
    client = commands.Bot(command_prefix='!', intents=discord.Intents.all())

    print("\n[client.tree.sync]")
    client.tree.sync()

    print("\n[client.get_channel]")
    clie

In [12]:
print(get_chat_response(chat, "解釋@pytest.fixture   用繁體中文回答"))

`@pytest.fixture` 裝飾器在 pytest 中用於宣告一個 fixture，這是一個回傳一個值並可供測試使用的函數。Fixture 通常用於提供共用測試資料或設定/銷毀資源。

Fixture 可以定義在模組或類別層級，並且可以被同一個模組或類別中的測試使用。它們被建立一次，然後在所有需要它們的測試中重複使用，這可以提升測試的效能，特別是在設定昂貴的資源或執行複雜計算時。

要在測試中使用 fixture，只需將它作為參數傳遞給測試函數即可。例如：

```python
@pytest.fixture
def my_fixture():
    return 10

def test_my_test(my_fixture):
    assert my_fixture == 10
```

在這個例子中，`my_fixture` 函數使用 `@pytest.fixture` 裝飾器宣告為一個 fixture。`test_my_test` 函數透過將 `my_fixture` 作為參數傳遞來使用它。

Fixture 也可以用於向其他 fixture 提供 fixture。例如：

```python
@pytest.fixture
def my_fixture1():
    return 10

@pytest.fixture
def my_fixture2(my_fixture1):
    return my_fixture1 + 10

def test_my_test(my_fixture2):
    assert my_fixture2 == 20
```

在這個例子中，`my_fixture2` fixture 依賴於 `my_fixture1` fixture。`my_fixture1` fixture 先被宣告，然後宣告 `my_fixture2` fixture 並將 `my_fixture1` 作為參數使用。`test_my_test` 函數然後使用 `my_fixture2` fixture 斷言其值為 20。

Fixture 也可以被參數化，這允許你使用不同的資料集來執行相同的測試。例如：

```python
@pytest.fixture(params=[1, 2, 3])
def my_fixture(reque